In [243]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.simplefilter("ignore")

In [244]:
result_df = pd.read_csv('./f1-data/results.csv')
stats_df = pd.read_csv('./f1-data/status.csv')
drivers_df = pd.read_csv('./f1-data/drivers.csv')
races_df = pd.read_csv('./f1-data/races.csv')
constructor_df = pd.read_csv('./f1-data/constructors.csv')
driver_standings_df = pd.read_csv('./f1-data/driver_standings.csv')
qualifying_df = pd.read_csv('./f1-data/qualifying.csv')
pd.get_option("display.max_columns",None)

20

In [245]:
con1 = pd.merge(result_df, qualifying_df, on = ['raceId', 'driverId'], copy = False, suffixes = ['_res', '_q'])
con2 = pd.merge(con1, races_df, on ='raceId', copy = False, suffixes = ['_con1', '_race'])
con3 = pd.merge(con2, drivers_df, on = 'driverId', copy = False, suffixes = ['_con2', '_driv'])
con4 = pd.merge(con3, driver_standings_df, on = ['driverId', 'raceId'], 
                copy = False, suffixes = ['_con3', '_driv_s'])

In [246]:
con4.columns

Index(['resultId', 'raceId', 'driverId', 'constructorId_res', 'number_res',
       'grid', 'position_res', 'positionText_con3', 'positionOrder',
       'points_con3', 'laps', 'time_con1', 'milliseconds', 'fastestLap',
       'rank', 'fastestLapTime', 'fastestLapSpeed', 'statusId', 'qualifyId',
       'constructorId_q', 'number_q', 'position_q', 'q1', 'q2', 'q3', 'year',
       'round', 'circuitId', 'name', 'date', 'time_race', 'url_con2',
       'driverRef', 'number', 'code', 'forename', 'surname', 'dob',
       'nationality', 'url_driv', 'driverStandingsId', 'points_driv_s',
       'position', 'positionText_driv_s', 'wins'],
      dtype='object')

In [247]:
con5 = pd.merge(con4, constructor_df, left_on ='constructorId_res', 
                right_on = 'constructorId', copy = False, suffixes = ['_con4', '_const'])
df = pd.merge(con5, stats_df, on ='statusId', copy = False, suffixes = ['_con5', '_stat'])
pd.get_option("display.max_columns",None)
df.head()

,resultId,raceId,driverId,constructorId_res,number_res,grid,position_res,positionText_con3,positionOrder,points_con3,...,points_driv_s,position,positionText_driv_s,wins,constructorId,constructorRef,name_const,nationality_const,url,status
0,1,18,1,1,22,1,1,1,1,10.0,...,10.0,1,1,1,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Finished
1,27,19,1,1,22,9,5,5,5,4.0,...,14.0,1,1,1,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Finished
2,69,21,1,1,22,5,3,3,3,6.0,...,20.0,2,2,1,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Finished
3,90,22,1,1,22,3,2,2,2,8.0,...,28.0,3,3,1,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Finished
4,109,23,1,1,22,3,1,1,1,10.0,...,38.0,1,1,2,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Finished


In [248]:
df.shape

(8954, 51)

In [249]:
df = df.drop(['raceId', 'constructorId_res', 'number_res', 'position_res', 'positionText_con3',
              'url_con2', 'url_driv', 'driverStandingsId'],1)

In [250]:
df = df.drop(['constructorId_q', 'constructorId', 'constructorRef', 'nationality_con4',
             'positionOrder', 'positionText_driv_s', 'time_con1', 'rank', 'statusId', 'resultId', 
             'driverId', 'qualifyId', 'number_q', 'round', 'circuitId', 'time_race', 'driverRef',
             'number', 'code'], 1)

In [251]:
df = df.drop(['grid', 'wins'], 1)

col_name = {'position_q':'starting_pos', 'milliseconds':'timetaken_in_millisec', 'fastestLapSpeed':'max_speed',
            'points_driv_s':'cumulative_points_including_this_race', 'name_const':'company', 'nationality_const':'nationality',
            'year':'race_year', 'name_con4':'race_track', 'position':'final_pos', 'points_con3':'points_from_this_race'}

df.rename(columns=col_name,inplace=True)
df.head()

,points_from_this_race,laps,timetaken_in_millisec,fastestLap,fastestLapTime,max_speed,starting_pos,q1,q2,q3,...,date,forename,surname,dob,cumulative_points_including_this_race,final_pos,company,nationality,url,status
0,10.0,58,5690616,39,1:27.452,218.300,1,1:26.572,1:25.187,1:26.714,...,16/03/08,Lewis,Hamilton,1985-01-07,10.0,1,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Finished
1,4.0,56,5525103,53,1:35.462,209.033,4,1:35.392,1:34.627,1:36.709,...,23/03/08,Lewis,Hamilton,1985-01-07,14.0,1,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Finished
2,6.0,66,5903238,20,1:22.017,204.323,5,1:21.366,1:20.825,1:22.096,...,27/04/08,Lewis,Hamilton,1985-01-07,20.0,2,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Finished
3,8.0,58,5213230,31,1:26.529,222.085,3,1:26.192,1:26.477,1:27.923,...,11/05/08,Lewis,Hamilton,1985-01-07,28.0,3,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Finished
4,10.0,76,7242742,71,1:18.510,153.152,3,1:15.582,1:15.322,1:15.839,...,25/05/08,Lewis,Hamilton,1985-01-07,38.0,1,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Finished


In [252]:
df.head(1).T

,0
points_from_this_race,10
laps,58
timetaken_in_millisec,5690616
fastestLap,39
fastestLapTime,1:27.452
max_speed,218.300
starting_pos,1
q1,1:26.572
q2,1:25.187
q3,1:26.714


In [253]:
df.head(100)

,points_from_this_race,laps,timetaken_in_millisec,fastestLap,fastestLapTime,max_speed,starting_pos,q1,q2,q3,...,date,forename,surname,dob,cumulative_points_including_this_race,final_pos,company,nationality,url,status
0,10.0,58,5690616,39,1:27.452,218.300,1,1:26.572,1:25.187,1:26.714,...,16/03/08,Lewis,Hamilton,1985-01-07,10.0,1,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Finished
1,4.0,56,5525103,53,1:35.462,209.033,4,1:35.392,1:34.627,1:36.709,...,23/03/08,Lewis,Hamilton,1985-01-07,14.0,1,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Finished
2,6.0,66,5903238,20,1:22.017,204.323,5,1:21.366,1:20.825,1:22.096,...,27/04/08,Lewis,Hamilton,1985-01-07,20.0,2,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Finished
3,8.0,58,5213230,31,1:26.529,222.085,3,1:26.192,1:26.477,1:27.923,...,11/05/08,Lewis,Hamilton,1985-01-07,28.0,3,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Finished
4,10.0,76,7242742,71,1:18.510,153.152,3,1:15.582,1:15.322,1:15.839,...,25/05/08,Lewis,Hamilton,1985-01-07,38.0,1,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Finished
5,0.0,70,5564783,40,1:17.453,205.022,3,1:15.634,1:15.293,1:16.693,...,22/06/08,Lewis,Hamilton,1985-01-07,38.0,4,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Finished
6,10.0,60,5949440,16,1:32.817,199.398,4,1:20.288,1:19.537,1:21.835,...,06/07/08,Lewis,Hamilton,1985-01-07,48.0,1,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Finished
7,10.0,67,5480874,17,1:16.039,216.552,1,1:15.218,1:14.603,1:15.666,...,20/07/08,Lewis,Hamilton,1985-01-07,58.0,1,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Finished
8,4.0,70,5870115,15,1:21.493,193.533,1,1:19.376,1:19.473,1:20.899,...,03/08/08,Lewis,Hamilton,1985-01-07,62.0,1,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Finished
9,8.0,57,5737950,16,1:38.884,197.285,2,1:38.464,1:37.954,1:39.199,...,24/08/08,Lewis,Hamilton,1985-01-07,70.0,1,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Finished


In [254]:
# df.to_csv('./cleaned-data.csv', index = False)

In [255]:
df['driver_name'] = df['forename']+' '+df['surname']
df = df.drop(['forename','surname'],1)
df['dob'] = pd.to_datetime(df['dob'])
df['date'] = pd.to_datetime(df['date'])
from datetime import datetime
dates = datetime.today()-df['dob']
age = dates.dt.days/365
df['age'] = round(age)

In [256]:
df['q1'] = pd.to_datetime(df['q1'], format='%M:%S.%f', errors='coerce')
df['q1'] = df.q1.dt.minute*60 + df.q1.dt.second 

df['q2'] = pd.to_datetime(df['q2'], format='%M:%S.%f', errors='coerce')
df['q2'] = df.q2.dt.minute*60 + df.q2.dt.second 

df['q3'] = pd.to_datetime(df['q3'], format='%M:%S.%f', errors='coerce')
df['q3'] = df.q3.dt.minute*60 + df.q3.dt.second

df['fastestLapTime'] = pd.to_datetime(df['fastestLapTime'], format='%M:%S.%f', errors='coerce')
df['fastestLapTime'] = df.fastestLapTime.dt.minute*60 + df.fastestLapTime.dt.second

In [257]:
df.head()

,points_from_this_race,laps,timetaken_in_millisec,fastestLap,fastestLapTime,max_speed,starting_pos,q1,q2,q3,...,date,dob,cumulative_points_including_this_race,final_pos,company,nationality,url,status,driver_name,age
0,10.0,58,5690616,39,87.0,218.300,1,86.0,85.0,86.0,...,2008-03-16,1985-01-07,10.0,1,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Finished,Lewis Hamilton,37.0
1,4.0,56,5525103,53,95.0,209.033,4,95.0,94.0,96.0,...,2008-03-23,1985-01-07,14.0,1,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Finished,Lewis Hamilton,37.0
2,6.0,66,5903238,20,82.0,204.323,5,81.0,80.0,82.0,...,2008-04-27,1985-01-07,20.0,2,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Finished,Lewis Hamilton,37.0
3,8.0,58,5213230,31,86.0,222.085,3,86.0,86.0,87.0,...,2008-11-05,1985-01-07,28.0,3,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Finished,Lewis Hamilton,37.0
4,10.0,76,7242742,71,78.0,153.152,3,75.0,75.0,75.0,...,2008-05-25,1985-01-07,38.0,1,McLaren,British,http://en.wikipedia.org/wiki/McLaren,Finished,Lewis Hamilton,37.0


In [258]:
#df['q1'][0].time()
#res = s.dt.strftime("%Y-%m-%d %H:%M:%S.%f")


In [259]:
#df['q1'].dt.strftime("%M:%S.%f")

In [260]:
df = df.drop(['url'], 1)

l = ['timetaken_in_millisec','fastestLap','max_speed']
for i in l:
    df[i] = pd.to_numeric(df[i],errors='coerce')

In [261]:
df.head(1).T

,0
points_from_this_race,10
laps,58
timetaken_in_millisec,5.69062e+06
fastestLap,39
fastestLapTime,87
max_speed,218.3
starting_pos,1
q1,86
q2,85
q3,86


In [262]:
df.dtypes

points_from_this_race                           float64
laps                                              int64
timetaken_in_millisec                           float64
fastestLap                                      float64
fastestLapTime                                  float64
max_speed                                       float64
starting_pos                                      int64
q1                                              float64
q2                                              float64
q3                                              float64
race_year                                         int64
race_track                                       object
date                                     datetime64[ns]
dob                                      datetime64[ns]
cumulative_points_including_this_race           float64
final_pos                                         int64
company                                          object
nationality                                     

In [263]:
df.isnull().sum() / len(df) * 100

points_from_this_race                     0.000000
laps                                      0.000000
timetaken_in_millisec                    56.622738
fastestLap                               22.481572
fastestLapTime                           22.481572
max_speed                                22.481572
starting_pos                              0.000000
q1                                        1.563547
q2                                       45.722582
q3                                       66.651776
race_year                                 0.000000
race_track                                0.000000
date                                      0.000000
dob                                       0.000000
cumulative_points_including_this_race     0.000000
final_pos                                 0.000000
company                                   0.000000
nationality                               0.000000
status                                    0.000000
driver_name                    

In [264]:
df.to_csv('./cleaned-data.csv', index = False)

In [265]:
df.head()

,points_from_this_race,laps,timetaken_in_millisec,fastestLap,fastestLapTime,max_speed,starting_pos,q1,q2,q3,...,race_track,date,dob,cumulative_points_including_this_race,final_pos,company,nationality,status,driver_name,age
0,10.0,58,5690616.0,39.0,87.0,218.300,1,86.0,85.0,86.0,...,Australian Grand Prix,2008-03-16,1985-01-07,10.0,1,McLaren,British,Finished,Lewis Hamilton,37.0
1,4.0,56,5525103.0,53.0,95.0,209.033,4,95.0,94.0,96.0,...,Malaysian Grand Prix,2008-03-23,1985-01-07,14.0,1,McLaren,British,Finished,Lewis Hamilton,37.0
2,6.0,66,5903238.0,20.0,82.0,204.323,5,81.0,80.0,82.0,...,Spanish Grand Prix,2008-04-27,1985-01-07,20.0,2,McLaren,British,Finished,Lewis Hamilton,37.0
3,8.0,58,5213230.0,31.0,86.0,222.085,3,86.0,86.0,87.0,...,Turkish Grand Prix,2008-11-05,1985-01-07,28.0,3,McLaren,British,Finished,Lewis Hamilton,37.0
4,10.0,76,7242742.0,71.0,78.0,153.152,3,75.0,75.0,75.0,...,Monaco Grand Prix,2008-05-25,1985-01-07,38.0,1,McLaren,British,Finished,Lewis Hamilton,37.0
